In [17]:
import csv
from pathlib import Path 
import numpy as np
from keras.preprocessing.text import Tokenizer
from sklearn.utils import shuffle
from keras.preprocessing import sequence
import os 

import tensorflow as tf

from keras import backend
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, Reshape
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline




In [7]:
ls

 Le volume dans le lecteur C s'appelle Windows
 Le num‚ro de s‚rie du volume est 460B-0FC5

 R‚pertoire de C:\Users\Maryam\Documents

15/05/2020  21:20    <DIR>          .
15/05/2020  21:20    <DIR>          ..
15/05/2020  21:16    <DIR>          .ipynb_checkpoints
30/04/2020  15:45    <DIR>          000A NON WINDOWS
04/05/2020  12:20    <DIR>          aclImdb
05/02/2019  20:07    <DIR>          Adobe
12/03/2019  10:59    <DIR>          Arduino
09/12/2018  22:39    <DIR>          Audacity
30/04/2020  15:45    <DIR>          Autre
07/10/2019  11:08    <DIR>          Blocs-notes OneNote
31/10/2018  13:25           230ÿ759 ContractEmploymentAgreement_ElMehdiCherradi_signed.pdf
23/03/2020  10:48    <DIR>          Downloads
04/04/2020  21:21    <DIR>          enigme
04/03/2019  14:47    <DIR>          LTspiceXVII
13/05/2020  13:52    <DIR>          MATLAB
01/11/2019  13:31    <DIR>          Matlab fichiers
04/10/2018  11:14    <DIR>          ModŠles Office personnalis‚s
14/05/2020  12:00   

In [3]:
max_features = 20000
maxlen = 100





Ytrain=[]
trainx=[]
Ytest=[]
testx=[]
print("loading the data...")
with open('out5.csv',encoding="utf8") as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        if csv_reader.line_num < 170001:
            Ytrain.append(int(row[0].split('/')[0]))
            trainx.append(row[0].split('\t')[1])
        elif csv_reader.line_num < 190001:
            Ytest.append(int(row[0].split('/')[0]))
            testx.append(row[0].split('\t')[1]) 
        else:
            break    


   

#194869


print("data loaded, processing data...")


# create the tokenizer
tok= Tokenizer(num_words=max_features, lower=True,filters='"#$%&()*+-/:<=>@[\\]^_`{|}~\t\n',split=' ', char_level=False, oov_token=None, document_count=0)
# fit the tokenizer on the documents
tok.fit_on_texts(trainx)
# summarize what was learned
#print(train.word_counts)
#print(train.document_count)
#print(train.word_index)
#print(train.word_docs)
# integer encode documents
Xtrain = tok.texts_to_sequences(trainx)

Xtest = tok.texts_to_sequences(testx)




#Xtrain, y_train = shuffle(Xtrain, Ytrain, random_state=0)
#Xtest, y_test = shuffle(Xtest, Ytest, random_state=0)


# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Ytrain)
encoded_Ytrain = encoder.transform(Ytrain)
encoded_Ytest = encoder.transform(Ytest)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Ytrain)
y_test = np_utils.to_categorical(encoded_Ytest)

encoder_length = len(y_train[0])
print("length of one hot enconding  ", encoder_length)

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(Xtrain, maxlen=maxlen)
x_test = sequence.pad_sequences(Xtest, maxlen=maxlen)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Ytrain = [x-1 for x in Ytrain]
Ytest = [x-1 for x in Ytest]



loading the data...
data loaded, processing data...
length of one hot enconding   10
Pad sequences (samples x time)
x_train shape: (169996, 100)
x_test shape: (20000, 100)


In [4]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt',encoding="utf8")
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((max_features, 100))
for word, i in tok.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


IndexError: index 20000 is out of bounds for axis 0 with size 20000

In [53]:
print(embedding_matrix)

[[-0.038194   -0.24487001  0.72812003 ... -0.1459      0.82779998
   0.27061999]
 [-0.071953    0.23127     0.023731   ... -0.71894997  0.86894
   0.19539   ]
 [-0.27085999  0.044006   -0.02026    ... -0.4923      0.63687003
   0.23642001]
 ...
 [ 0.16046999  0.86734998 -0.085039   ... -0.71705002 -0.15491
   0.30443001]
 [ 0.015885   -0.17775001  1.09720004 ... -0.26109999 -0.34308001
   0.44424999]
 [ 0.10107    -0.90408999  0.89310002 ... -0.71322    -0.08908
   0.10901   ]]


In [66]:

# define model
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping

embedding_size = 100


model = Sequential()
e = Embedding(max_features,embedding_size,weights=[embedding_matrix], input_length=maxlen,trainable=False)
model.add(e)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(encoder_length, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

epochs = 5
batch_size = 64


history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 100, 100)          2000000   
_________________________________________________________________
spatial_dropout1d_10 (Spatia (None, 100, 100)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1010      
Total params: 2,081,410
Trainable params: 81,410
Non-trainable params: 2,000,000
_________________________________________________________________
None
Train on 152996 samples, validate on 17000 samples
Epoch 1/5
152996/152996 [==============================] - 527s 3ms/step - loss: 1.8482 - acc: 0.3266 - val_loss: 1.7442 - val_acc: 0.3452
Epoch 2/5
152996/152996 [==============================] - 520s 

In [65]:
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.metrics import top_k_categorical_accuracy


embedding_size = 100

def top2_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

model2 = Sequential()
model2.add(Embedding(max_features,embedding_size, input_length=maxlen))
model2.add(SpatialDropout1D(0.2))
model2.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(encoder_length, activation='softmax'))

model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 3
batch_size = 64


history2 = model2.fit(x_train, Ytrain, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



score, acc = model2.evaluate(x_test, Ytest, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


Train on 152996 samples, validate on 17000 samples
Epoch 1/3
  6080/152996 [>.............................] - ETA: 12:55 - loss: 2.0652 - acc: 0.2757 - sparse_categorical_accuracy: 0.2757

KeyboardInterrupt: 

In [5]:

# define model
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping

embedding_size = 100


model = Sequential()
e = Embedding(max_features,embedding_size,weights=[embedding_matrix], input_length=maxlen,trainable=False)
model.add(e)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(encoder_length, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

epochs = 10
batch_size = 64


history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          2000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 100, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 2,081,410
Trainable params: 81,410
Non-trainable params: 2,000,000
_________________________________________________________________
None
Train on 152996 samples, validate on 17000 samples
Epoch 1/10
152996/152996 [==============================] - 325s 2ms/step - loss: 1.8882 - acc: 0.3163 - val_loss: 1.7754 - val_acc: 0.3407
Epoch 2/10
152996/152996 [==============================] - 316

In [6]:

# define model
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping

embedding_size = 100


model = Sequential()
e = Embedding(max_features,embedding_size,weights=[embedding_matrix], input_length=maxlen,trainable=False)
model.add(e)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(encoder_length, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

epochs = 20
batch_size = 64


history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          2000000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 100, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 2,081,410
Trainable params: 81,410
Non-trainable params: 2,000,000
_________________________________________________________________
None
Train on 152996 samples, validate on 17000 samples
Epoch 1/20
152996/152996 [==============================] - 324s 2ms/step - loss: 1.8812 - acc: 0.3192 - val_loss: 1.8268 - val_acc: 0.3243
Epoch 2/20
152996/152996 [==============================] - 329

In [23]:
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.metrics import top_k_categorical_accuracy 

embedding_size = 100

model = Sequential()
e = Embedding(max_features,embedding_size,weights=[embedding_matrix], input_length=maxlen,trainable=False)
model.add(e)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(encoder_length, activation='softmax'))

model.compile(loss='mse', optimizer='adam')

epochs = 2
batch_size = 64

#Ytrain = [x-1 for x in Ytrain]
#Ytest = [x-1 for x in Ytest]


history2 = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


Train on 152996 samples, validate on 17000 samples
Epoch 1/2
152996/152996 [==============================] - 361s 2ms/step - loss: 0.0803 - val_loss: 0.0776
Epoch 2/2
20000/20000 [==============================] - 14s 682us/step


TypeError: 'numpy.float64' object is not iterable

In [24]:

score = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)


20000/20000 [==============================] - 11s 557us/step
Test score: 0.07526018905639649


In [26]:
new_review = ['Oh my God ! This movie was awesome, just perfect, I could not think of anything better. All the scenes were just so nice to watch']
seq = tok.texts_to_sequences(new_review)
padded = sequence.pad_sequences(seq, maxlen=maxlen)
pred = model.predict(padded)
labels = [1,2,3,4,5,6,7,8,9,10]
print(pred, labels[np.argmax(pred)])

[[0.14509894 0.03687759 0.03119751 0.01546801 0.02140452 0.02866917
  0.05456748 0.09557567 0.12362949 0.44751167]] 10


In [27]:
new_review = ['What a stupid movie. I really did not like it, it was very bad and the director did the worse job of his whole career']
seq = tok.texts_to_sequences(new_review)
padded = sequence.pad_sequences(seq, maxlen=maxlen)
pred = model.predict(padded)
labels = [1,2,3,4,5,6,7,8,9,10]
print(pred, labels[np.argmax(pred)])

[[0.5646696  0.10991977 0.10340281 0.0578124  0.03570674 0.02500441
  0.02539086 0.02639755 0.0181576  0.03353834]] 1


In [28]:
new_review = ["I am quite mitigated, I don't know what to think of that movie. It was good and bad at the same time. Surprising and predictible."]
seq = tok.texts_to_sequences(new_review)
padded = sequence.pad_sequences(seq, maxlen=maxlen)
pred = model.predict(padded)
labels = [1,2,3,4,5,6,7,8,9,10]
print(pred, labels[np.argmax(pred)])

[[0.17072645 0.05555867 0.05578766 0.03812353 0.04418208 0.04738629
  0.09064323 0.12582459 0.11485077 0.25691676]] 10


In [25]:
model.save("with_embedding_0")

In [29]:
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.metrics import top_k_categorical_accuracy 

embedding_size = 100

model = Sequential()
model.add(Embedding(max_features,embedding_size, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(encoder_length, activation='softmax'))

model.compile(loss='mse', optimizer='adam')

epochs = 2
batch_size = 64

#Ytrain = [x-1 for x in Ytrain]
#Ytest = [x-1 for x in Ytest]


history2 = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


Train on 152996 samples, validate on 17000 samples
Epoch 1/2
152996/152996 [==============================] - 448s 3ms/step - loss: 0.0755 - val_loss: 0.0735
Epoch 2/2
20000/20000 [==============================] - 13s 636us/step


TypeError: 'numpy.float64' object is not iterable

In [31]:
score= model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)

20000/20000 [==============================] - 11s 564us/step
Test score: 0.07158285917043686


In [33]:
model.save('without_embedding_0')

In [34]:
new_review = ['Oh my God ! This movie was awesome, just perfect, I could not think of anything better. All the scenes were just so nice to watch']
seq = tok.texts_to_sequences(new_review)
padded = sequence.pad_sequences(seq, maxlen=maxlen)
pred = model.predict(padded)
labels = [1,2,3,4,5,6,7,8,9,10]
print(pred, labels[np.argmax(pred)])

[[0.03288252 0.01065903 0.00577785 0.00667291 0.00762039 0.0096976
  0.02153268 0.0400192  0.10282385 0.76231396]] 10


In [35]:
new_review = ['What a stupid movie. I really did not like it, it was very bad and the director did the worse job of his whole career']
seq = tok.texts_to_sequences(new_review)
padded = sequence.pad_sequences(seq, maxlen=maxlen)
pred = model.predict(padded)
labels = [1,2,3,4,5,6,7,8,9,10]
print(pred, labels[np.argmax(pred)])

[[0.6464631  0.12189397 0.08601493 0.03376023 0.02737855 0.02094109
  0.01476607 0.01635375 0.00951275 0.02291551]] 1


In [36]:
new_review = ["I am quite mitigated, I don't know what to think of that movie. It was good and bad at the same time. Surprising and predictible."]
seq = tok.texts_to_sequences(new_review)
padded = sequence.pad_sequences(seq, maxlen=maxlen)
pred = model.predict(padded)
labels = [1,2,3,4,5,6,7,8,9,10]
print(pred, labels[np.argmax(pred)])

[[0.06163768 0.04243037 0.04357338 0.05787824 0.06657224 0.06494371
  0.14852828 0.13370521 0.20629641 0.17443444]] 9


In [37]:
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping

embedding_size = 100

model = Sequential()
model.add(Embedding(max_features,embedding_size, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(encoder_length, activation='softmax'))

model.compile(loss='mse', optimizer='sgd')

epochs = 2
batch_size = 64

#Ytrain = [x-1 for x in Ytrain]
#Ytest = [x-1 for x in Ytest]


history2 = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



Train on 152996 samples, validate on 17000 samples
Epoch 1/2
152996/152996 [==============================] - 464s 3ms/step - loss: 0.0894 - val_loss: 0.0889
Epoch 2/2
152996/152996 [==============================] - 476s 3ms/step - loss: 0.0881 - val_loss: 0.0878


In [38]:
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)

20000/20000 [==============================] - 12s 596us/step
Test score: 0.08762401130199432


In [41]:
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

from keras.layers import Flatten
from keras.callbacks import EarlyStopping

embedding_size = 100

model = Sequential()
e = Embedding(max_features,embedding_size,weights=[embedding_matrix], input_length=maxlen,trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(encoder_length,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

epochs = 2
batch_size = 64


history2 = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


Train on 152996 samples, validate on 17000 samples
Epoch 1/2
152996/152996 [==============================] - 18s 118us/step - loss: 1.9246 - acc: 0.2996 - val_loss: 1.9012 - val_acc: 0.3038
Epoch 2/2
20000/20000 [==============================] - 1s 53us/step
Test score: 1.8610599002838135
Test accuracy: 0.31305


In [42]:
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

from keras.layers import Flatten
from keras.callbacks import EarlyStopping

embedding_size = 100

model = Sequential()
e = Embedding(max_features,embedding_size,input_length=maxlen)
model.add(e)
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(encoder_length,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

epochs = 2
batch_size = 64


history2 = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


Train on 152996 samples, validate on 17000 samples
Epoch 1/2
152996/152996 [==============================] - 121s 792us/step - loss: 1.6439 - acc: 0.3838 - val_loss: 1.6164 - val_acc: 0.3934
Epoch 2/2
20000/20000 [==============================] - 3s 138us/step
Test score: 1.757355754852295
Test accuracy: 0.3752
